In [ ]:
import implicit
import numpy as np
import pandas as pd
import surprise as srp
import scipy.sparse as sparse
import data_management as dm

from implicit.nearest_neighbours import bm25_weight
from implicit.datasets.lastfm import get_lastfm

## Загрузка данных

In [2]:
user_event_df = dm.get_user_event_dataframe(
    user_event_file_path='../dataset/cliks_add.csv'
)
user_event_df

,user_id,event_id,event_name,clicks_count
0,00001deaca68c3e7a9943ad6d29d1a95,1205186,Концерт оркестра UralBand,2
1,00001deaca68c3e7a9943ad6d29d1a95,1237397,Выставка «Антика. Западноевропейское искусство...,1
2,00001deaca68c3e7a9943ad6d29d1a95,1264968,Выставка «От романтизма к импрессионизму. Фран...,1
3,00002fa07c02629bf0dc022a6c0283d1,476106,Выставка «Искусство ХХ века»,1
4,000061466abdfa67ca963ae752fa937d,185644,Мюзикл «Снежная королева»,1
...,...,...,...,...
630584,ffff5daad033e2b9b57593627d9b1a9b,1189248,Выставка «91 день счастья»,2
630585,ffff5daad033e2b9b57593627d9b1a9b,1390909,Музыкальная программа The Beatles,2
630586,ffff5daad033e2b9b57593627d9b1a9b,1423074,Ночь искусств в Волгоградском музее изобразите...,1
630587,ffff63c5a29c432e2ca2fb4aa968d7ba,1292376,Посещение Павловского парка,1


In [3]:
users_df = dm.get_user_dataframe(
    users_file_path='../dataset/users_and_purchases/users.txt',
    regions_file_path='../dataset/users_and_purchases/region.txt',
    regions_nums_file_path='../dataset/RegionRussia.csv'
)
users_df

,user_id,region,region_code,age
0,9af020ef757c476c64fbdf0b71781dfe,Республика Коми,11,22.0
1,06d589eba089ca6bcb984d32c5399cc5,Пермский край,59,22.0
2,2816bd3aabf33fbc25c82678571b0e81,Краснодарский край,23,23.0
3,f6fcc2dadfeb783308bd102ddcbc1412,Саратовская область,64,17.0
4,e0b3e50d8d9f467470a8aa2b42899841,Москва,77,21.0
...,...,...,...,...
2231047,6e8f4dc75245926306665942fd1173b2,Ханты-Мансийский автономный округ – Югра,86,14.0
2231048,c925794714279efbdbe2e5cba0de7b28,Москва,77,19.0
2231049,18cf6bb59c25e9c6493f89780348b622,Ханты-Мансийский автономный округ – Югра,86,18.0
2231050,ec8d3c862c1383c63f8adbca559bf86c,Ростовская область,61,20.0


In [4]:
events_df = dm.get_events_dataframe(
    events_file_path='../dataset/users_and_purchases/events/events_pushka_accepted_30122021.csv',
    organizations_file_path='../dataset/users_and_purchases/events/organizations.csv'
)
events_df

,event_id,event_name,link,add_link,org_id,org_name,region_name,category
0,36,Спектакль «Касатка»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6,ОГАУК «Драматический театр имени М. С. Щепкина»,Белгородская обл,Театры
1,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6,ОГАУК «Драматический театр имени М. С. Щепкина»,Белгородская обл,Театры
2,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6,ОГАУК «Драматический театр имени М. С. Щепкина»,Белгородская обл,Театры
3,139,Спектакль «Куклы»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6,ОГАУК «Драматический театр имени М. С. Щепкина»,Белгородская обл,Театры
4,150,Спектакль «Филумена Мартурано»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6,ОГАУК «Драматический театр имени М. С. Щепкина»,Белгородская обл,Театры
...,...,...,...,...,...,...,...,...
47851,1518521,Опера «Садко»,https://tickets.mariinsky.ru/ru/performance/VG...,NaN,5268,ФГБУК «Государственный академический Мариински...,г Санкт-Петербург,Театры
47852,1518585,Опера «Фауст»,https://tickets.mariinsky.ru/ru/performance/Nk...,NaN,5268,ФГБУК «Государственный академический Мариински...,г Санкт-Петербург,Театры
47853,1518645,Опера «Трубадур»,https://tickets.mariinsky.ru/ru/performance/OW...,NaN,5268,ФГБУК «Государственный академический Мариински...,г Санкт-Петербург,Театры
47854,1519214,Спектакль «Царевна-лягушка»,https://ulan-ude.kassy.ru/events/range-2021-12/,NaN,12276,АУК РБ Бурятский республиканский театр кукол «...,Респ Бурятия,Театры


In [5]:
user_event = dm.filter_user_event_by_user_region(
    user_event_df=user_event_df,
    user_df=users_df,
    user_region=77
)
user_event.shape

(40148, 4)

In [6]:
user_event = dm.filter_user_event_by_event_region(
    user_event_df=user_event,
    event_df=events_df,
    event_region='г Москва'
)
user_event.shape


(35557, 4)

## Модель для Пушкинской карты
### Alternative Least Squares

In [7]:
user_event['event_id'] = user_event['event_id'].astype(int)
user_event['event_id'] = user_event['event_id'].astype('category')
user_event['user_id'] = user_event['user_id'].astype('category')
user_event['user_num'] = user_event['user_id'].cat.codes
user_event['event_id'] = user_event['event_id'].cat.codes
user_event

,user_id,event_id,event_name,clicks_count,user_num
13,0001ad0041738ac671fe52a5f887742e,337,Экспозиция «Древнерусская иконопись XIII–XVI в...,1,0
14,0001ad0041738ac671fe52a5f887742e,592,Экспозиция музея Владимира Высоцкого,1,0
15,0001ad0041738ac671fe52a5f887742e,882,Экспозиции Мультимедиа Арт Музея,1,0
16,0001ad0041738ac671fe52a5f887742e,1481,Выставка «За все тебя благодарю»,1,0
17,0001ad0041738ac671fe52a5f887742e,1654,Вечер музыки П. И. Чайковского,1,0
...,...,...,...,...,...
630238,ffe13ffdd93b045e3b06ee834c585d26,1150,Спектакль «Дневник Анны Франк. Белая роза»,1,13655
630274,ffe4a90226a335c9ed184bc19163490b,513,Спектакль «Анна Каренина»,1,13656
630275,ffe4a90226a335c9ed184bc19163490b,1145,Спектакль «Люди как люди»,1,13656
630276,ffe4a90226a335c9ed184bc19163490b,1151,Спектакль «Мнимый больной»,1,13656


In [8]:
sparse_event_user = sparse.csr_matrix(
    (user_event['clicks_count'].astype(float), (user_event['event_id'], user_event['user_num']))
)
sparse_user_event = sparse.csr_matrix(
    (user_event['clicks_count'].astype(float), (user_event['user_num'], user_event['event_id']))
)

In [20]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

alpha_val = 15
data_conf = (sparse_event_user * alpha_val).astype('double')

model.fit(data_conf)


  0%|          | 0/20 [00:00<?, ?it/s]

In [35]:
user_num = 0
recommended = model.recommend(user_num, sparse_user_event)

events = list()
scores = list()
print(recommended)
for item in recommended:
    idx, score = item
    events.append(user_event.event_name.loc[user_event.event_id == idx].iloc[0])
    scores.append(score)

recommendations = pd.DataFrame({
    'event_name': events,
    'score': scores
})

[(1650, 0.6320194), (165, 0.6068883), (1289, 0.5776259), (1039, 0.55948097), (487, 0.547245), (328, 0.52730536), (562, 0.50552773), (1413, 0.5046071), (1415, 0.48967427), (1653, 0.47331524)]


#### Что кликал пользователь:

In [11]:
visited = user_event.loc[user_event.user_num == 0]['event_name']
visited

13    Экспозиция «Древнерусская иконопись XIII–XVI в...
14                 Экспозиция музея Владимира Высоцкого
15                     Экспозиции Мультимедиа Арт Музея
16                     Выставка «За все тебя благодарю»
17                       Вечер музыки П. И. Чайковского
Name: event_name, dtype: object

#### Рекомендации для пользователя:

In [22]:
recommendations

,event_name,score
0,Концерт «Вечера органной музыки»,0.632019
1,Выставка «Шедевры русского искусства XI — нача...,0.606888
2,Цикл научно-популярных лекций «Анатомия чувств»,0.577626
3,"Цикл концертов «Мама, я меломан»",0.559481
4,Мюзикл «Шахматы»,0.547245
5,Выставка «Карадаг заповедный»,0.527305
6,Экспозиция картинной галереи Ильи Глазунова,0.505528
7,Выставка «Частоты 2.0»,0.504607
8,Экскурсия «Профессия – реставратор»,0.489674
9,Концерт Дениса Мацуева и Томского академическо...,0.473315


## Bayesian Personalized Ranking

In [13]:
bpr_model = implicit.bpr.BayesianPersonalizedRanking(factors=60)
bpr_model.fit(sparse_event_user)

  0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
user_id = '0001ad0041738ac671fe52a5f887742e'
user_num = user_event['user_num'].loc[user_event['user_id'] == user_id]
recommended = bpr_model.recommend(user_num.iloc[0], sparse_user_event)

events = list()
scores = list()

for item in recommended:
    idx, score = item
    events.append(user_event.event_name.loc[user_event.event_id == idx].iloc[0])
    scores.append(score)

bpr_recommendations = pd.DataFrame({
    'event_name': events,
    'score': scores
})

0


#### Что кликал пользователь:

In [15]:
visited = user_event.loc[user_event.user_num == 0]['event_name']
visited

13    Экспозиция «Древнерусская иконопись XIII–XVI в...
14                 Экспозиция музея Владимира Высоцкого
15                     Экспозиции Мультимедиа Арт Музея
16                     Выставка «За все тебя благодарю»
17                       Вечер музыки П. И. Чайковского
Name: event_name, dtype: object

#### Рекомендации для пользователя:

In [16]:
bpr_recommendations

,event_name,score
0,Выставка «Карадаг заповедный»,1.265012
1,Выставка «Частоты 2.0»,1.252784
2,Экскурсия «Профессия – реставратор»,1.247311
3,Экспозиции Большого дворца и Хлебного дома,1.240108
4,"Цикл концертов «Мама, я меломан»",1.184246
5,Выставка «Да живет иное во мне»,1.179139
6,Экскурсии по выставке «Бывают странные сближен...,1.165090
7,Основная экспозиция Исторического музея,1.160802
8,Экскурсия по выставке «1960-е. Москва космичес...,1.109341
9,Экспозиция картинной галереи Ильи Глазунова,1.106534
